In [2]:

import os
import json

import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session
role = get_execution_role()
s3 = boto3.client("s3")
sess = Session()


In [4]:
# setups

import os
import json

import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session

role = get_execution_role()
s3 = boto3.client("s3")
sess = Session()
pt_model_data = sess.upload_data(
        path="model.tar.gz", bucket=sess.default_bucket(), key_prefix="model"
    )
# # Get global config
# with open("code/config.json", "r") as f:
#     CONFIG = json.load(f)

# sess = Session()

# role = get_execution_role()

# %store -r pt_mnist_model_data

# try:
#     pt_mnist_model_data
# except NameError:
#     import json

#     # copy a pretrained model from a public public to your default bucket
#     s3 = boto3.client("s3")
#     bucket = CONFIG["public_bucket"]
#     key = "datasets/image/MNIST/model/pytorch-training-2020-11-21-22-02-56-203/model.tar.gz"
#     s3.download_file(bucket, key, "model.tar.gz")

#     # upload to default bucket
#     pt_mnist_model_data = sess.upload_data(
#         path="model.tar.gz", bucket=sess.default_bucket(), key_prefix="model/pytorch"
#     )

In [5]:
pt_model_data

's3://sagemaker-us-east-1-368703438116/model/model.tar.gz'

In [6]:
model = PyTorchModel(
    entry_point="inference.py",
    source_dir="code",
    role=role,
    model_data=pt_model_data,
    framework_version="1.5.0",
    py_version="py3",
)



In [7]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# set local_mode to False if you want to deploy on a remote
# SageMaker instance

local_mode = False

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.c4.xlarge"

# predictor = model.deploy(
#     initial_instance_count=1,
#     instance_type=instance_type,
#     #serializer=JSONSerializer(),
#     #deserializer=JSONDeserializer(),
# )
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
#     serializer=JSONSerializer(),
#     deserializer=JSONDeserializer(),
)

-------!

In [257]:
import json
import os
import re
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification
from transformers import BertModel
from transformers import BertTokenizer

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence


import re
# from model import Model
# from utils import collate_batch


# data_path='data/prediction_0717.csv'
batch_size=32
# init_from_ckpt="ckpt_ernie_gram_bs8_lr5e-5_epoch25/epoch_9.pth"
# device="cuda"
import os
import json
from transformers import BertTokenizer, BertModel
import re

import jieba
import pandas as pd
from torch.utils.data import Dataset


In [396]:
prompt = "Working with SageMaker makes machine learning "
json.dumps(prompt)invoke_endpoint( 

'"Working with SageMaker makes machine learning "'

In [309]:
tags_file = os.path.join("data/tags.txt") #not sure
with open(tags_file, "r", encoding="utf8") as f:
        label_dict = json.load(f)
        inv_label = {value: key for key, value in label_dict.items()}
#pred_ds = MyDataSet(input_data, is_test=True) ###改为input_fn(request_body)   


def collate_batch(batch, tokenizer, label_dict=None, is_test=False):
    text_a, text_b, labels = [], [], []
    for a, b, part, label in batch:
        text_a.append(part + " " + b)
        text_b.append(part + " " + a)
    encoded_inputs = tokenizer(text=text_a, text_pair=text_b, padding=True, return_tensors="pt")
    if not is_test and isinstance(label_dict, dict):
        labels = torch.tensor([label_dict[item[3]] for item in batch])
        return encoded_inputs["input_ids"], encoded_inputs["token_type_ids"], encoded_inputs["attention_mask"], labels
    else:
        return encoded_inputs["input_ids"], encoded_inputs["token_type_ids"], encoded_inputs["attention_mask"]
    
 

In [112]:
class MyDataSet(Dataset):

    def __init__(self, file_path, is_test=False):
        self.is_test = is_test
        self.data = pd.read_csv(file_path)[0:1]
        #self.data = pd.read_csv(file_path)

    def __getitem__(self, idx):
        if self.is_test:
            text_a, text_b, part = self.data.loc[idx]["Defect Found"], self.data.loc[idx][
                "Work Executed"], self.data.loc[idx]['QM Part Structure Text']
            part = part.strip().lower()
            if not (isinstance(text_a, str)):
                text_a = ""
            if not (isinstance(text_b, str)):
                text_b = ""
            text_a = re.sub(r"sj[0-9A-Za-z]*[-/]*\d*|SJ[0-9A-Za-z]*[-/]*\d*|//\d*-|\d*-\d*-", "", text_a)
            text_b = re.sub(r"</br>|br|\sbr\s|\s</br>\s", "。", text_b)
            return text_a, text_b, part, None
        else:
            return self.data.loc[idx]["text_a"], self.data.loc[idx]["text_b"],\
                   self.data.loc[idx]["part"], self.data.loc[idx]["label"]

    def __len__(self):
        return self.data.shape[0]
   

In [323]:
pred_ds = MyDataSet("data/prediction_0717.csv", is_test=True)
tokenizer = BertTokenizer.from_pretrained("nghuyong/ernie-gram-zh")
trans_fn = partial(collate_batch, tokenizer=tokenizer, label_dict=label_dict, is_test=True)
data_loader = DataLoader(pred_ds, shuffle=False, batch_size=batch_size, collate_fn=trans_fn)

###----------------------------
test1 = []
test2=[]
test3=[]
with torch.no_grad():
        for idx, (input_ids, token_type_ids, attention_mask) in enumerate(data_loader):
            test1=input_ids
            test2=token_type_ids
            test3=attention_mask

# ###------------------------------
# input_data={}
# input_data["input_ids"]=test1.tolist()
# input_data["token_type_ids"]=test2.tolist()
# input_data["attention_mask"]=test3.tolist()

In [434]:
text1="E代码: , FD:  ,序列号:  ,  问题描述:  SJ236I01JC;微信报单, 备注:[期望上门时间:2022/06/19 上午（9:00-12:00）] E代码:SJ236I01JC,问题描述: 微信报单,显示E/F故障代码, 备注:前天显示e14 水龙头亮灯今天显示e14 太阳图标亮灯2次都没洗完,优先联系电话：电话-13257008798,微信授权手机号：13184590362 (期望上门时间:2022/06/19 上午（9:00-12:00）)   显示故障代码/故障信号6"
text2="我看到:显示屏显示E14触点故障 br 我做了:更换流量开关br 备注:"
text3="WATER LEVEL SYSTEM"
input_data= {"t1": text1,"t2":text2,"t3":text3}

In [435]:
input_data

{'t1': 'E代码: , FD:  ,序列号:  ,  问题描述:  SJ236I01JC;微信报单, 备注:[期望上门时间:2022/06/19 上午（9:00-12:00）] E代码:SJ236I01JC,问题描述: 微信报单,显示E/F故障代码, 备注:前天显示e14 水龙头亮灯今天显示e14 太阳图标亮灯2次都没洗完,优先联系电话：电话-13257008798,微信授权手机号：13184590362 (期望上门时间:2022/06/19 上午（9:00-12:00）)   显示故障代码/故障信号6',
 't2': '我看到:显示屏显示E14触点故障 br 我做了:更换流量开关br 备注:',
 't3': 'WATER LEVEL SYSTEM'}

In [439]:
ppp='{"t1": "E代码: , FD:  ,序列号:  ,  问题描述:  SJ236I01JC;微信报单, 备注:[期望上门时间:2022/06/19 上午（9:00-12:00）] E代码:SJ236I01JC,问题描述: 微信报单,显示E/F故障代码, 备注:前天显示e14 水龙头亮灯今天显示e14 太阳图标亮灯2次都没洗完,优先联系电话：电话-13257008798,微信授权手机号：13184590362 (期望上门时间:2022/06/19 上午（9:00-12:00）)   显示故障代码/故障信号6","t2":"我看到:显示屏显示E14触点故障 br 我做了:更换流量开关br 备注:","t3":"WATER LEVEL SYSTEM"}'
ppp



'{"t1": "E代码: , FD:  ,序列号:  ,  问题描述:  SJ236I01JC;微信报单, 备注:[期望上门时间:2022/06/19 上午（9:00-12:00）] E代码:SJ236I01JC,问题描述: 微信报单,显示E/F故障代码, 备注:前天显示e14 水龙头亮灯今天显示e14 太阳图标亮灯2次都没洗完,优先联系电话：电话-13257008798,微信授权手机号：13184590362 (期望上门时间:2022/06/19 上午（9:00-12:00）)   显示故障代码/故障信号6","t2":"我看到:显示屏显示E14触点故障 br 我做了:更换流量开关br 备注:","t3":"WATER LEVEL SYSTEM"}'

In [511]:
payload = '{"t1": "E代码: , FD:  ,序列号:  ,  问题描述:  SJ236I01JC;微信报单, 备注:[期望上门时间:2022/06/19 上午（9:00-12:00）] E代码:SJ236I01JC,问题描述: 微信报单,显示E/F故障代码, 备注:前天显示e14 水龙头亮灯今天显示e14 太阳图标亮灯2次都没洗完,优先联系电话：电话-13257008798,微信授权手机号：13184590362 (期望上门时间:2022/06/19 上午（9:00-12:00）)   显示故障代码/故障信号6","t2":"我看到:显示屏显示E14触点故障 br 我做了:更换流量开关br 备注:","t3":"WATER LEVEL SYSTEM"}'

client = boto3.client("sagemaker-runtime")

response = client.invoke_endpoint(
    EndpointName="pytorch-inference-2022-12-14-03-10-55-023", ContentType="application/json", Body=payload
)

In [513]:
output = response["Body"].read()
output

b'[["water level system-e14"], [0.9979701638221741]]'

In [466]:
import boto3
payload = '{"t1": "E代码: , FD:  ,序列号:  ,  问题描述:  SJ236I01JC;微信报单, 备注:[期望上门时间:2022/06/19 上午（9:00-12:00）] E代码:SJ236I01JC,问题描述: 微信报单,显示E/F故障代码, 备注:前天显示e14 水龙头亮灯今天显示e14 太阳图标亮灯2次都没洗完,优先联系电话：电话-13257008798,微信授权手机号：13184590362 (期望上门时间:2022/06/19 上午（9:00-12:00）)   显示故障代码/故障信号6","t2":"我看到:显示屏显示E14触点故障 br 我做了:更换流量开关br 备注:","t3":"WATER LEVEL SYSTEM"}'

client = boto3.client("sagemaker-runtime")

response = client.invoke_endpoint(
    EndpointName="pytorch-inference-2022-12-13-15-56-57-197", ContentType="application/json", Body=payload
)

In [467]:
output = response["Body"].read()

In [519]:
output=bytes.decode(output)

In [525]:
data=json.loads(output)

In [527]:
data[1]

[0.9979701638221741]